In [ ]:
# 1. 克隆官方仓库
!git clone https://github.com/yaolu/ordered-prompt.git

# 2. 进入项目目录
%cd ordered-prompt

# 3. 安装依赖库
# 论文代码主要依赖 transformers 和 torch
# 为了兼容性，建议安装一般版本，如果有报错再指定版本

!pip install transformers scikit-learn numpy scipy torch pyyaml

Cloning into 'ordered-prompt'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 133 (delta 39), reused 36 (delta 36), pack-reused 86 (from 1)
Receiving objects: 100% (133/133), 28.54 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/ordered-prompt


In [ ]:
# 假设仓库中有 data 文件夹和下载脚本
# 如果没有脚本，通常通过 HuggingFace datasets 库自动下载
# 这里尝试运行仓库自带的数据脚本（如果有的话）
!bash scripts/download_data.sh

# 或者，您可以手动上传数据：
# 点击左侧文件夹图标 -> 直接拖拽您的 sst-2 数据集文件到 ordered-prompt/data/ 目录下

bash: scripts/download_data.sh: No such file or directory


In [ ]:
# 查看仓库里的文件列表，找到真正的 python 主程序名字
!ls -R /content/ordered-prompt

/content/ordered-prompt:
augment.py	dataset.py   gpt3_augument.py	  gpt3_utils.py  model.py
compeletion.py	debugger.py  gpt3_compeletion.py  __init__.py	 README.md
config		entropy.py   gpt3_prediction.py   LICENSE	 script
data		evaluate.py  gpt3.py		  main.py	 utils.py

/content/ordered-prompt/config:
agnews.yaml  cr.yaml	   mpqa.yaml  rte.yaml	 sst5.yaml  trec.yaml
cb.yaml      dbpedia.yaml  mr.yaml    sst2.yaml  subj.yaml

/content/ordered-prompt/data:
agnews	cb  cr	dbpedia  mpqa  mr  rte	sst2  sst5  subj  trec

/content/ordered-prompt/data/agnews:
classes.txt  dev_subsample.jsonl  test.jsonl  train.jsonl

/content/ordered-prompt/data/cb:
dev.jsonl  dev_subsample.jsonl	test.jsonl  train.jsonl

/content/ordered-prompt/data/cr:
dev_subsample.jsonl  test.jsonl  train.jsonl

/content/ordered-prompt/data/dbpedia:
classes.txt  dev_subsample.jsonl  readme.txt  test.jsonl  train_subset.jsonl

/content/ordered-prompt/data/mpqa:
dev_subsample.jsonl  test.jsonl  train.jsonl

/content/ordered-pr

In [ ]:
# 运行主程序
# --model_name_or_path: 使用 huggingface 上的 "gpt2" (0.1B参数)
# --task_name: 任务名称，比如 sst2
# --num_shots: 4 (论文核心设置)
# --method: probe (使用论文提出的探测方法)

!python main.py \
    --model_name_or_path gpt2 \
    --task_name sst2 \
    --num_shots 4 \
    --method probe \
    --probe_metric global_entropy \
    --output_dir ./output_results

2025-12-09 13:42:32.172248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765287752.191663    2733 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765287752.197586    2733 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765287752.212509    2733 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765287752.212532    2733 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765287752.212536    2733 computation_placer.cc:177] computation placer alr

In [ ]:
import os

# 1. 尝试定位 main.py 所在的真正路径
target_dir = "/content/ordered-prompt"
if not os.path.exists(os.path.join(target_dir, "main.py")):
    # 或者是这一层
    target_dir = "/content/ordered-prompt/ordered-prompt"

# 2. 进入该目录
os.chdir(target_dir)
print(f"当前工作目录: {os.getcwd()}")

# 3. 再次确认文件是否存在
!ls -F main.py config/ script/

当前工作目录: /content/ordered-prompt
main.py

config/:
agnews.yaml  cr.yaml	   mpqa.yaml  rte.yaml	 sst5.yaml  trec.yaml
cb.yaml      dbpedia.yaml  mr.yaml    sst2.yaml  subj.yaml

script/:
gpt3/	       run_cb.sh  run_dbpedia.sh  run_mr.sh   run_sst2.sh  run_subj.sh
run_agnews.sh  run_cr.sh  run_mpqa.sh	  run_rte.sh  run_sst5.sh  run_trec.sh


In [ ]:
!cat script/run_sst2.sh

#!/bin/bash

MAIN_DIR=$(pwd)
DATASET=sst2
LOGDIR=experiment_32shot/$DATASET;
SAMPLE_MODE=balance

mkdir -p $LOGDIR

for NSHOT in 4;
do

for MODEL in gpt2 gpt2-medium gpt2-large gpt2-xl;
do
for SEED in 1 2 3 4 5;
do

  for N in 3 5;
  do
     python main.py --config config/$DATASET.yaml \
     --nshot $NSHOT --model $MODEL --output $LOGDIR --seed $SEED \
     --ngram $N --generate --temperature 2.0 --topk 20 --do_sample

     echo "python main.py --config config/$DATASET.yaml \
     --nshot $NSHOT --model $MODEL --output $LOGDIR --seed $SEED \
     --ngram $N --generate --temperature 2.0 --topk 20 --do_sample"
  done;

  cd $LOGDIR;

  for f in generate*;
  do
      python "$MAIN_DIR"/augment.py $f
  done;

  mkdir ckpt;
  mv *.pkl ckpt;

  OUTPUT=dev_${DATASET}_${NSHOT}_shot_${MODEL}_seed${SEED}.jsonl
  cat augment_*.jsonl > $OUTPUT
  mv augment_*.jsonl ckpt

  cd "${MAIN_DIR}" || exit;

  echo "python main.py --config config/$DATASET.yaml \
     --nshot $NSHOT --model $MODEL --output 

In [ ]:
import os
import glob
import shutil

# --- 配置参数 (模拟脚本中的变量) ---
MAIN_DIR = os.getcwd() # 当前目录，应该是 /content/ordered-prompt
DATASET = "sst2"
NSHOT = 4
MODEL = "gpt2"         # 为了速度先用 gpt2，跑通后再换 gpt2-xl
SEED = 1               # 只跑种子 1
LOGDIR = f"experiment_32shot/{DATASET}" # 保持和脚本一致的目录结构

# 创建实验目录
os.makedirs(LOGDIR, exist_ok=True)

print(f"🚀 开始复现流程: Model={MODEL}, Task={DATASET}, Seed={SEED}")
print("-" * 50)

# ==========================================
# 步骤 1: 生成探测集 (Generate Probing Set)
# ==========================================
print("Step 1: 生成探测数据 (Generating)...")
# 脚本里循环了 ngram 3 和 5，我们照做
for ngram in [3, 5]:
    cmd = (
        f"python main.py --config config/{DATASET}.yaml "
        f"--nshot {NSHOT} --model {MODEL} --output {LOGDIR} --seed {SEED} "
        f"--ngram {ngram} --generate --temperature 2.0 --topk 20 --do_sample"
    )
    print(f"执行: {cmd}")
    os.system(cmd)

# ==========================================
# 步骤 2: 数据清洗与合并 (Augment)
# ==========================================
print("\nStep 2: 清洗数据 (Augmenting)...")
# 找到生成的文件 (generate*)
gen_files = glob.glob(os.path.join(LOGDIR, "generate*"))

for f in gen_files:
    # 调用 augment.py 处理每个生成的文件
    cmd = f"python augment.py {f}"
    print(f"执行: {cmd}")
    os.system(cmd)

# 整理文件 (把中间文件移到 ckpt 文件夹，防止混乱)
ckpt_dir = os.path.join(LOGDIR, "ckpt")
os.makedirs(ckpt_dir, exist_ok=True)

# 移动 .pkl 文件
for pkl in glob.glob("*.pkl"):
    shutil.move(pkl, ckpt_dir)

# 合并所有 augment_*.jsonl 到一个文件
output_filename = f"dev_{DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}.jsonl"
output_path = os.path.join(LOGDIR, output_filename)
aug_files = glob.glob("augment_*.jsonl")

with open(output_path, 'w') as outfile:
    for fname in aug_files:
        with open(fname) as infile:
            outfile.write(infile.read())
        # 移动处理完的碎片文件到 ckpt
        shutil.move(fname, ckpt_dir)

print(f"生成的探测集已合并至: {output_path}")

# ==========================================
# 步骤 3: 在探测集上推理 (Inference on Fake Data)
# ==========================================
print("\nStep 3: 计算探测集表现 (Inference on Probing Set)...")
# 注意：这里指定了 --test_data_path 为我们刚刚生成的假数据
cmd = (
    f"python main.py --config config/{DATASET}.yaml "
    f"--nshot {NSHOT} --model {MODEL} --output {LOGDIR} --seed {SEED} "
    f"--test_data_path {output_path}"
)
print(f"执行: {cmd}")
os.system(cmd)

# 重命名结果文件为 fake_...pkl
# 脚本逻辑：main.py 会生成一个类似 {DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}_xxxx.pkl 的文件
# 我们需要找到最新的那个 pkl 并重命名
res_files = glob.glob(os.path.join(LOGDIR, f"{DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}_*.pkl"))
# 假设只有一个或最新的一个是目标文件
if res_files:
    latest_file = max(res_files, key=os.path.getctime)
    fake_pkl = os.path.join(LOGDIR, f"fake_{DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}.pkl")
    os.rename(latest_file, fake_pkl)
    print(f"Fake 结果已保存: {fake_pkl}")
else:
    print("❌ 错误: 未找到 Fake 推理结果 pkl 文件")

# ==========================================
# 步骤 4: 在真实验证集上推理 (Inference on True Data)
# ==========================================
print("\nStep 4: 计算真实数据表现 (Inference on True Data)...")
# 这次不带 --test_data_path，默认用真实验证集
cmd = (
    f"python main.py --config config/{DATASET}.yaml "
    f"--nshot {NSHOT} --model {MODEL} --output {LOGDIR} --seed {SEED}"
)
print(f"执行: {cmd}")
os.system(cmd)

# 重命名结果文件为 true_...pkl
res_files = glob.glob(os.path.join(LOGDIR, f"{DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}_*.pkl"))
if res_files:
    latest_file = max(res_files, key=os.path.getctime)
    true_pkl = os.path.join(LOGDIR, f"true_{DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}.pkl")
    os.rename(latest_file, true_pkl)
    print(f"True 结果已保存: {true_pkl}")
else:
    print("❌ 错误: 未找到 True 推理结果 pkl 文件")

# ==========================================
# 步骤 5: 计算熵并输出结果 (Entropy Selection)
# ==========================================
print("\nStep 5: 计算熵并筛选最佳 Prompt (Entropy Calculation)...")
result_json = os.path.join(LOGDIR, f"result_{DATASET}_{NSHOT}_shot_{MODEL}_seed{SEED}.json")

cmd = (
    f"python entropy.py --true {true_pkl} "
    f"--fake {fake_pkl} "
    f"--topk 4 --save {result_json}"
)
print(f"执行: {cmd}")
os.system(cmd)

print("\n" + "="*50)
print(f"✅ 复现完成！请检查结果文件: {result_json}")
print("你可以打开该 JSON 文件查看筛选出的最佳 Prompt 顺序及其效果。")

🚀 开始复现流程: Model=gpt2, Task=sst2, Seed=1
--------------------------------------------------
Step 1: 生成探测数据 (Generating)...
执行: python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output experiment_32shot/sst2 --seed 1 --ngram 3 --generate --temperature 2.0 --topk 20 --do_sample
执行: python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output experiment_32shot/sst2 --seed 1 --ngram 5 --generate --temperature 2.0 --topk 20 --do_sample

Step 2: 清洗数据 (Augmenting)...
生成的探测集已合并至: experiment_32shot/sst2/dev_sst2_4_shot_gpt2_seed1.jsonl

Step 3: 计算探测集表现 (Inference on Probing Set)...
执行: python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output experiment_32shot/sst2 --seed 1 --test_data_path experiment_32shot/sst2/dev_sst2_4_shot_gpt2_seed1.jsonl
❌ 错误: 未找到 Fake 推理结果 pkl 文件

Step 4: 计算真实数据表现 (Inference on True Data)...
执行: python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output experiment_32shot/sst2 --seed 1
❌ 错误: 未找到 True 推理结果 pkl 文件

S

NameError: name 'true_pkl' is not defined

In [ ]:
# 1. 强制进入代码仓库目录
%cd /content/ordered-prompt

# 2. 再次确认文件确实在这里 (应该能看到 main.py)
!ls

/content/ordered-prompt
augment.py	entropy.py	     gpt3.py	    ordered-prompt
compeletion.py	evaluate.py	     gpt3_utils.py  __pycache__
config		experiment_32shot    __init__.py    README.md
data		gpt3_augument.py     LICENSE	    script
dataset.py	gpt3_compeletion.py  main.py	    utils.py
debugger.py	gpt3_prediction.py   model.py


In [ ]:
# 运行 Step 4: 在真实数据上评估 (不带 --test_data_path 参数)
# 这一步会直接读取 data/sst2/ 下的文件，验证模型是否工作正常

!python main.py \
    --config config/sst2.yaml \
    --nshot 4 \
    --model gpt2 \
    --output experiment_32shot/sst2 \
    --seed 1

2025-12-09 13:52:43.071531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765288363.090557    5456 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765288363.096473    5456 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765288363.112120    5456 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765288363.112143    5456 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765288363.112148    5456 computation_placer.cc:177] computation placer alr

In [ ]:
import os

# 定义参数
LOGDIR = "experiment_32shot/sst2"
# 确保目录存在
os.makedirs(LOGDIR, exist_ok=True)

print("🚀 正在重新尝试生成探测集 (Probing Set)...")

# 运行生成命令 (只跑 ngram=3 以节省时间)
# 注意：我加了打印，方便看它到底生成了什么文件
cmd = (
    "python main.py "
    "--config config/sst2.yaml "
    "--nshot 4 "
    "--model gpt2 "
    "--output experiment_32shot/sst2 "
    "--seed 1 "
    "--ngram 3 "
    "--generate "         # 关键参数：开启生成模式
    "--temperature 2.0 "  # 关键参数：高温度增加多样性
    "--topk 20 "
    "--do_sample"
)

print(f"执行命令: {cmd}")
exit_code = os.system(cmd)

if exit_code == 0:
    print("\n✅ 生成命令执行成功！正在检查生成的文件...")
    # 检查目录下有没有 generate 开头的文件
    files = os.listdir(LOGDIR)
    gen_files = [f for f in files if f.startswith("generate")]

    if gen_files:
        print(f"🎉 成功找到生成文件: {gen_files}")
        print("下一步：请把这些乱七八糟的生成文件清洗成标准格式。")

        # 立即运行清洗脚本
        print("\n🧹 正在清洗数据 (Augment)...")
        for f in gen_files:
            full_path = os.path.join(LOGDIR, f)
            os.system(f"python augment.py {full_path}")

        print("✅ 清洗完成！你应该能看到 augment_ 开头的 jsonl 文件了。")
    else:
        print("⚠️ 命令跑通了，但没找到 'generate' 开头的文件。请检查上方日志。")
else:
    print("❌ 生成命令失败！请查看上方的报错日志。")

🚀 正在重新尝试生成探测集 (Probing Set)...
执行命令: python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output experiment_32shot/sst2 --seed 1 --ngram 3 --generate --temperature 2.0 --topk 20 --do_sample

✅ 生成命令执行成功！正在检查生成的文件...
🎉 成功找到生成文件: ['generate_3gram_sst2_4_shot_gpt2_seed1_balance_temperature2.0_top20_hashd12673.pkl']
下一步：请把这些乱七八糟的生成文件清洗成标准格式。

🧹 正在清洗数据 (Augment)...
✅ 清洗完成！你应该能看到 augment_ 开头的 jsonl 文件了。


In [ ]:
import os
import glob
import shutil
import sys

# ============================
# 1. 强制修正工作目录
# ============================
# 检查当前是否在 ordered-prompt 目录，如果不是就进去
current_dir = os.getcwd()
if not current_dir.endswith("ordered-prompt"):
    print(f"⚠️ 当前目录是 {current_dir}，正在切换到 /content/ordered-prompt ...")
    try:
        os.chdir("/content/ordered-prompt")
    except FileNotFoundError:
        print("❌ 找不到 /content/ordered-prompt 目录！请确认你是否克隆了代码。")
        sys.exit(1)
else:
    print(f"✅ 当前目录正确: {current_dir}")

# ============================
# 2. 暴力寻找并修复数据文件
# ============================
LOGDIR = "experiment_32shot/sst2"
TARGET_FILENAME = "dev_sst2_4_shot_gpt2_seed1.jsonl"
TARGET_PATH = os.path.join(LOGDIR, TARGET_FILENAME)

print("\n🕵️‍♂️ 正在全盘搜索 augment 文件...")

# 策略：哪里有augment文件，就抓过来改名
# 1. 先找子文件夹
candidates_subdir = glob.glob(os.path.join(LOGDIR, "augment_*.jsonl"))
# 2. 再找根目录
candidates_root = glob.glob("augment_*.jsonl")
# 3. 看看目标文件是不是已经存在了
target_exists = os.path.exists(TARGET_PATH)

if target_exists:
    print(f"✅ 目标文件已存在: {TARGET_PATH} (直接使用)")

elif candidates_subdir:
    src = candidates_subdir[0]
    print(f"👉 在子目录找到了: {src}")
    shutil.move(src, TARGET_PATH)
    print(f"✅ 已重命名为: {TARGET_PATH}")

elif candidates_root:
    src = candidates_root[0]
    print(f"👉 在根目录找到了: {src}")
    shutil.move(src, TARGET_PATH)
    print(f"✅ 已移动并重命名为: {TARGET_PATH}")

else:
    print("❌ 严重错误：到处都找不到 augment_*.jsonl 文件！")
    print("请检查左侧文件栏，experiment_32shot/sst2/ 目录下到底有没有生成过 jsonl 文件？")
    # 如果真的没了，可能需要重跑生成步骤，这里先中止
    sys.exit(1)

# ============================
# 3. 继续运行 Step 3, 4, 5
# ============================
print("\n🚀 文件修复完成，开始跑最后流程！")

# Step 3: 假数据评分
print("--- Step 3: Fake Data Eval ---")
exit_code = os.system(f"python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output {LOGDIR} --seed 1 --test_data_path {TARGET_PATH}")
if exit_code != 0: raise RuntimeError("Step 3 运行失败")

# 整理 Step 3 的结果 (fake pkl)
fake_candidates = glob.glob(os.path.join(LOGDIR, "sst2_4_shot_gpt2_seed1_*.pkl"))
if not fake_candidates: raise RuntimeError("Step 3 没生成 pkl 结果")
latest_fake = max(fake_candidates, key=os.path.getctime)
FAKE_PKL = os.path.join(LOGDIR, "fake_scores.pkl")
os.rename(latest_fake, FAKE_PKL)

# Step 4: 真数据评分 (检查是否需要补跑)
print("--- Step 4: True Data Eval ---")
TRUE_PKL = os.path.join(LOGDIR, "true_scores.pkl")
# 检查之前是否已经生成过 true_scores
if not os.path.exists(TRUE_PKL):
    # 找一下有没有没改名的
    true_candidates = [f for f in glob.glob(os.path.join(LOGDIR, "sst2_4_shot_gpt2_seed1_*.pkl")) if "fake" not in f]
    if true_candidates:
        shutil.move(true_candidates[0], TRUE_PKL)
        print("✅ 找到旧的真实评分文件，已重命名。")
    else:
        print("🔄 没找到真实评分文件，正在补跑 Step 4...")
        os.system(f"python main.py --config config/sst2.yaml --nshot 4 --model gpt2 --output {LOGDIR} --seed 1")
        # 抓取新生成的文件
        true_candidates = [f for f in glob.glob(os.path.join(LOGDIR, "sst2_4_shot_gpt2_seed1_*.pkl")) if "fake" not in f]
        if true_candidates:
            shutil.move(true_candidates[0], TRUE_PKL)
        else:
            raise RuntimeError("Step 4 运行失败，没生成 pkl")

# Step 5: 计算熵
print("--- Step 5: Final Entropy Calculation ---")
FINAL_JSON = os.path.join(LOGDIR, "final_result.json")
os.system(f"python entropy.py --true {TRUE_PKL} --fake {FAKE_PKL} --topk 4 --save {FINAL_JSON}")

if os.path.exists(FINAL_JSON):
    import json
    print("\n" + "="*50)
    print("🎆 复现成功！Best Prompt 如下：")
    with open(FINAL_JSON, 'r') as f:
        print(json.dumps(json.load(f), indent=2))
else:
    print("❌ 流程结束，但没生成 JSON，请查看上方报错。")

✅ 当前目录正确: /content/ordered-prompt

🕵️‍♂️ 正在全盘搜索 augment 文件...
👉 在子目录找到了: experiment_32shot/sst2/augment_generate_3gram_sst2_4_shot_gpt2_seed1_balance_temperature2.0_top20_hashd12673.jsonl
✅ 已重命名为: experiment_32shot/sst2/dev_sst2_4_shot_gpt2_seed1.jsonl

🚀 文件修复完成，开始跑最后流程！
--- Step 3: Fake Data Eval ---
--- Step 4: True Data Eval ---
🔄 没找到真实评分文件，正在补跑 Step 4...
--- Step 5: Final Entropy Calculation ---

🎆 复现成功！Best Prompt 如下：
{
  "acc_stats": [
    0.5672041666666666,
    0.06688639298072684
  ],
  "topk_acc_stats": [
    0.6543,
    0.07380945061440303
  ],
  "topk": 4,
  "entropys": [
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    0.43291507546338787,
    0.689734492670935,
    1.5783735610181542e-06,
    1.5783735610181542e-06,
    0.28499889328570227,
    1.5783735610181542e-06,
    0.2623322442988144,
    0.6465905855361525,
    1.57